<a href="https://colab.research.google.com/github/otubrempong/CNN/blob/main/TFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch


In [2]:
import pandas as pd
import numpy as np
import torch


In [132]:
data = pd.read_csv('/content/synthetic_mobility_dataset_D2D.csv')

In [133]:
from sklearn.pipeline import Pipeline

In [146]:
data.head()

,timeStamps,deviceIDs,signalStrengths,speeds,latitudes,longitudes,userActivities,previousConnectedDeviceID
0,00:00:00,20,-66,0,40.479994,-73.872472,Stationary,0
1,00:00:08,41,-84,10,40.499247,-73.647203,Running,0
2,00:00:17,46,-65,0,40.681396,-73.770962,Stationary,0
3,00:00:25,30,-62,0,40.620723,-73.305220,Stationary,0
4,00:00:34,44,-57,8,40.856452,-73.942643,Running,0


In [134]:
data['timeStamps'] = data['timeStamps'].str.split().str[1]


In [123]:
data.timeStamps = data.timeStamps.str.strip('01-Jan-2023')

In [147]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   timeStamps                 20000 non-null  object 
 1   deviceIDs                  20000 non-null  int64  
 2   signalStrengths            20000 non-null  int64  
 3   speeds                     20000 non-null  int64  
 4   latitudes                  20000 non-null  float64
 5   longitudes                 20000 non-null  float64
 6   userActivities             20000 non-null  object 
 7   previousConnectedDeviceID  20000 non-null  int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 1.2+ MB


In [10]:
from sklearn.preprocessing import StandardScaler

In [125]:
categorical_columns = ['userActivities']

In [126]:
timestamp_column = ['timeStamps']

In [ ]:
df['timeStamps'] = pd.to_datetime(df['timeStamps'], format='%H:%M:%S')

In [142]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pandas as pd

class ConvertToDatetime(BaseEstimator, TransformerMixin):
    def __init__(self, timestamp_column):
        self.timestamp_column = timestamp_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X[self.timestamp_column] = pd.to_datetime(X[self.timestamp_column],format='%H:%M:%S').dt.time
        return X

class HandleMissingValues(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.num_imputer = SimpleImputer(strategy='median')
        self.cat_imputer = SimpleImputer(strategy='most_frequent')

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for col in X.columns:
            if X[col].dtype == 'object':  # Categorical column
                X[col] = self.cat_imputer.fit_transform(X[[col]])
            else:  # Numerical column
                X[col] = self.num_imputer.fit_transform(X[[col]])
        return X
from sklearn.preprocessing import LabelEncoder

class EncodeCategorical(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_columns):
        self.categorical_columns = categorical_columns
        self.encoders = {col: LabelEncoder() for col in self.categorical_columns}

    def fit(self, X, y=None):
        for col in self.categorical_columns:
            self.encoders[col].fit(X[col])
        return self

    def transform(self, X):
        for col in self.categorical_columns:
            X[col] = self.encoders[col].transform(X[col])
        return X
class StandardScaleNumerical(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
        X[numerical_cols] = self.scaler.fit_transform(X[numerical_cols])
        return X

In [ ]:


pipeline = Pipeline(steps=[
    ('handle_missing_values', HandleMissingValues()),
    ('encode_categorical', EncodeCategorical(categorical_columns)),
    ('standard_scale_numerical', StandardScaleNumerical()),
])

# Apply the pipeline
data_transformed = pipeline.fit_transform(data)

In [145]:
data['timeStamps'] = pd.to_datetime(data['timeStamps'], format='%H:%M:%S').dt.time

In [129]:
data_transformed

,timeStamps,deviceIDs,signalStrengths,speeds,latitudes,longitudes,userActivities,previousConnectedDeviceID
0,00:00:,-0.396480,-0.045379,-0.683441,-0.077624,-1.282582,0.707743,-1.764118
1,00:00:08,1.063648,-0.920352,-0.354220,-0.010849,-0.501545,0.000707,-1.764118
2,00:00:17,1.411297,0.003230,-0.683441,0.620903,-0.930632,0.707743,-1.764118
3,00:00:25,0.298819,0.149059,-0.683441,0.410468,0.684161,0.707743,-1.764118
4,00:00:34,1.272237,0.392107,-0.420065,1.228054,-1.525877,0.000707,-1.764118
...,...,...,...,...,...,...,...,...
19995,23:59:25,0.855058,-1.017572,-0.255454,1.356831,1.733672,-1.413365,0.246308
19996,23:59:34,0.576938,0.100449,-0.518831,1.432180,1.212454,1.414780,1.147534
19997,23:59:4,-1.022250,-0.482866,-0.090844,1.227112,0.656570,-1.413365,-0.862892
19998,23:59:5,0.507408,0.732374,1.686950,0.297203,1.257925,-0.706329,-0.169642


In [63]:
import pandas as pd

# Assuming df is your DataFrame and it has 'latitude' and 'longitude' columns
data_transformed['latitudes'], latitude_bins = pd.cut(data_transformed['latitudes'], bins=10, labels=False, retbins=True)
data_transformed['longitudes'], longitude_bins = pd.cut(data_transformed['longitudes'], bins=10, labels=False, retbins=True)

# Combine the discretized latitude and longitude values into a single 'location' column
data_transformed['location'] = list(zip(data_transformed['latitudes'], data_transformed['longitudes']))

In [64]:
data_transformed.head()

,timeStamps,deviceIDs,signalStrengths,speeds,latitudes,longitudes,userActivities,previousConnectedDeviceID,location
0,00:00:,-0.396480,-0.045379,-0.683441,4,1,0.707743,-1.764118,"(4, 1)"
1,00:00:08,1.063648,-0.920352,-0.354220,4,3,0.000707,-1.764118,"(4, 3)"
2,00:00:17,1.411297,0.003230,-0.683441,6,2,0.707743,-1.764118,"(6, 2)"
3,00:00:25,0.298819,0.149059,-0.683441,6,6,0.707743,-1.764118,"(6, 6)"
4,00:00:34,1.272237,0.392107,-0.420065,8,0,0.000707,-1.764118,"(8, 0)"


In [69]:
data_transformed= data_transformed.drop(['latitudes', 'longitudes'], axis=1)

In [70]:
data_transformed.head()

,timeStamps,deviceIDs,signalStrengths,speeds,userActivities,previousConnectedDeviceID,location
0,00:00:,-0.396480,-0.045379,-0.683441,0.707743,-1.764118,"(4, 1)"
1,00:00:08,1.063648,-0.920352,-0.354220,0.000707,-1.764118,"(4, 3)"
2,00:00:17,1.411297,0.003230,-0.683441,0.707743,-1.764118,"(6, 2)"
3,00:00:25,0.298819,0.149059,-0.683441,0.707743,-1.764118,"(6, 6)"
4,00:00:34,1.272237,0.392107,-0.420065,0.000707,-1.764118,"(8, 0)"


In [72]:
pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 33.4 MB/s eta 0:00:00


In [74]:
pip install pytorch_forecasting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.8/155.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.7 MB/s eta 0:00:00


In [76]:
pip install --upgrade pytorch-forecasting

In [78]:
pip install --upgrade pytorch-forecasting pytorch-lightning optuna


In [80]:
import pandas as pd
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, QuantileLoss


In [81]:
data_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   timeStamps                 20000 non-null  object 
 1   deviceIDs                  20000 non-null  float64
 2   signalStrengths            20000 non-null  float64
 3   speeds                     20000 non-null  float64
 4   userActivities             20000 non-null  float64
 5   previousConnectedDeviceID  20000 non-null  float64
 6   location                   20000 non-null  object 
dtypes: float64(5), object(2)
memory usage: 1.1+ MB


In [131]:
data.loc[0,'timeStamps'] = '00:00:00'

In [106]:
data_transformed['timeStamps'] = data_transformed['timeStamps'].str.strip()  # Remove leading/trailing spaces


In [96]:
data_transformed.loc[0,'timeStamps'] == ' 00:00: '

False

In [107]:
print(data['timeStamps'].head(10))


0       00:00:
1     00:00:08
2     00:00:17
3     00:00:25
4     00:00:34
5      00:00:4
6      00:00:5
7       00:01:
8     00:01:09
9     00:01:17
Name: timeStamps, dtype: object
